In [1]:
import pandas as pd
import os
import indian_state_list as isl
import tkinter as tk
import indian_map_image_clor as in_map

d_state = isl.d_state
path = os.getcwd()

database = pd.read_csv(path + '/year_record_filtered/record_2015_filtered.csv')
database = database.iloc[:,1:]
database.head()

SUBDIVISION  YEAR   JAN    FEB    MAR     APR    MAY    JUN    JUL  \
0     andhra pradesh  2015   2.0   0.60   5.50   32.30   34.1  283.8  116.0   
1  arunachal pradesh  2015  30.8  47.50  97.50  287.10  238.9  637.9  329.3   
2              assam  2015  13.4  15.50  37.50  250.90  332.5  558.5  300.1   
3        west bengal  2015  14.3  10.25  42.05  118.85  181.1  377.7  513.2   
4              bihar  2015  12.8   1.80  27.20   38.70   39.5  122.1  231.5   

     AUG     SEP    OCT    NOV    DEC   ANNUAL     JF     MAM     JJAS     OND  
0  192.0  201.80  59.70  81.20   2.00  1010.90   2.50   71.90   793.60  142.80  
1  595.5  374.20  65.20  33.80  29.80  2767.50  78.30  623.50  1936.90  128.80  
2  590.9  279.90  62.60  14.00  15.20  2470.90  28.90  620.90  1729.30   91.80  
3  443.6  259.45  43.15  13.05   7.65  2024.45  24.55  342.05  1593.95   63.85  
4  287.0  101.70  10.40   0.00   0.00   872.70  14.60  105.50   742.30   10.40

In [2]:
def get_value_month(state,month) :
    d = database[database.SUBDIVISION == state].copy()
    return d.iloc[:,month]

def get_max_value_month(month) :
    return database.iloc[:,month].max()

def convert_to_color(state,month):
    month = month + 1
    max_v = get_max_value_month(month)
    div = 10
    element = max_v // div
    val = get_value_month(state,month)
    c = val // element
    color = (int(250 - (200 / div * c)), 0, 0)
    return color

def create_color_list(d_state,month):
    color_list = []
    for x in d_state.keys():
        color_list.append(convert_to_color(x,month))
    return color_list

def create_color_rbg_2_hex(div):
    l = []
    for c in range(div):
        r,g,b = (int(250 - (200 // div * c)), 0, 0)
        l.append("#{:02x}{:02x}{:02x}".format(r,g,b))
    return l

print(create_color_list(d_state,1))

print(create_color_rbg_2_hex(10))

[(230, 0, 0), (130, 0, 0), (250, 0, 0), (170, 0, 0), (150, 0, 0), (250, 0, 0), (250, 0, 0), (250, 0, 0), (210, 0, 0), (210, 0, 0), (230, 0, 0), (210, 0, 0), (150, 0, 0), (210, 0, 0), (210, 0, 0), (210, 0, 0), (30, 0, 0), (70, 0, 0), (210, 0, 0), (210, 0, 0), (250, 0, 0), (210, 0, 0), (210, 0, 0), (210, 0, 0), (210, 0, 0), (210, 0, 0), (210, 0, 0), (250, 0, 0), (210, 0, 0)]
['#fa0000', '#e60000', '#d20000', '#be0000', '#aa0000', '#960000', '#820000', '#6e0000', '#5a0000', '#460000']


In [3]:
window = tk.Tk()

w = 1000
h = 750

month_selected = 1
map_drawn = ""

map_IN = None

def create_color_rbg_2_hex(div):
    l = []
    for c in range(div):
        b,g,r = (int(250 - (200 // div * c)), 0, 0)
        l.append("#{:02x}{:02x}{:02x}".format(r,g,b))
    return l

def draw_map_given(month) :
    return in_map.create_map_india_with_state(create_color_list(d_state,month))

def select_month_btns(event):
    global month_selected,map_drawn,map_IN,parent_canvas,img
    case = event.x // 50 + 1
    if event.y > 50 :
        case = case + 6
    print(case)
    if month_selected == case :
        print("Same Selection")
    else :
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#dddddd", outline="")
        month_selected = case
        canvas_month_sel.itemconfig('month_btn' + str(month_selected), fill="#666666", outline="")
        map_drawn = draw_map_given(case)
        img = tk.PhotoImage(file=map_drawn, master=parent_canvas)
        parent_canvas.itemconfig('map_IN' ,image=img)
        update_legends(get_max_value_month(case+1))

def update_legends(max_v) :
    element_val = max_v / 10
    for x in range(10):
        canvas_legends.itemconfig( 'legend_txt'+str(x), text=" value less Than " + str(round(((x + 1) * element_val),2)))
        
# map resolution is constant 662 x 750
parent_canvas = tk.Canvas(window, width=w, height=h)

canvas_month_sel = tk.Canvas(parent_canvas, width=310, height=200)
for x in range(6):
    btn1 = canvas_month_sel.create_oval( 10 + x * 50, 10, 50 + x * 50, 50, fill="#dddddd", outline="", tags="month_btn" + str(x + 1))
    txt1 = canvas_month_sel.create_text( 30 + x * 50, 28, text=(x + 1), font='Time 20 bold', fill="#ffffff", tags="month_txt" + str(x + 1))
    canvas_month_sel.tag_bind(btn1, '<Button-1>', select_month_btns)
    canvas_month_sel.tag_bind(txt1, '<Button-1>', select_month_btns)
    btn2 = canvas_month_sel.create_oval( 10 + x * 50, 60, 50 + x * 50, 100, fill="#dddddd", outline="", tags="month_btn" + str(x + 7))
    txt2 = canvas_month_sel.create_text( 30 + x * 50, 78, text=(x + 7), font='Time 20 bold', fill="#ffffff", tags="month_txt" + str(x + 7))
    canvas_month_sel.tag_bind(btn2, '<Button-1>', select_month_btns)
    canvas_month_sel.tag_bind(txt2, '<Button-1>', select_month_btns)
else :
    canvas_month_sel.itemconfig('month_btn1', fill="#666666", outline="")
    map_drawn = draw_map_given(1)
parent_canvas.create_window(675,100, anchor=tk.NW, window=canvas_month_sel)

canvas_legends = tk.Canvas(parent_canvas, width=200, height=300)
parent_canvas.create_window(450,440, anchor=tk.NW, window=canvas_legends)
legend_color = create_color_rbg_2_hex(10)
element_val = get_max_value_month(2)/10
print(get_max_value_month(2))
for x in range(10):
    canvas_legends.create_line(10, 20 + x * 30 ,30, 20 + x * 30, width=20, fill=legend_color[x])
    canvas_legends.create_text(110, 20 + x * 30, text= " value less Than " + str(round(((x + 1) * element_val),2)), font='Time 14', fill="#666666", tags="legend_txt"+str(x) )

img = tk.PhotoImage(file=map_drawn, master=parent_canvas)
map_IN = parent_canvas.create_image(0, 0, image=img, anchor = tk.NW, tags="map_IN")
# text = tk.Entry(parent_canvas, width=5)
# parent_canvas.create_window((700,100), window=text, anchor='nw')
parent_canvas.pack()
tk.mainloop()

750 662
[255 255 255]
67.2
2
750 662
[255 255 255]
3
750 662
[255 255 255]
10
750 662
[255 255 255]
10
Same Selection
10
Same Selection
